In [1]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-3.1.1-py3-none-any.whl pytorch-tabnet
!pip install /kaggle/input/iterative-stratification/iterative-stratification-master/

Looking in links: /kaggle/input/pytorchtabnet/pytorch_tabnet-3.1.1-py3-none-any.whl
Processing /kaggle/input/pytorchtabnet/pytorch_tabnet-3.1.1-py3-none-any.whl
Processing /kaggle/input/iterative-stratification/iterative-stratification-master
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for iterative-stratification: filename=iterative_stratification-0.1.6-py3-none-any.whl size=8401 sha256=fbe4da5fb960459ae72b0349355df68ffe2b6551938f243b7e175a732391f0c9
  Stored in directory: /root/.cache/pip/wheels/b8/47/3f/eb4af42d124f37d23d6f13a4c8bbc32c1d70140e6e1cecb4aa
Successfully built iterative-stratification


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_drug.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/test_features.csv
/kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
/kaggle/input/pytorchtabnet/pytorch_tabnet-3.1.0-py3-none-any.whl
/kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.1-py3-none-any.whl
/kaggle/input/pytorchtabnet/pytorch_tabnet-1.2.0-py3-none-any.whl
/kaggle/input/pytorchtabnet/pytorch_tabnet-3.1.1-py3-none-any.whl
/kaggle/input/pytorchtabnet/pytorch_tabnet-3.0.0-py3-none-any.whl
/kaggle/input/iterative-stratification/iterative-stratification-master/.travis.yml
/kaggle/input/iterative-stratification/iterative-stratification-master/setup.cfg
/kaggle/input/iterative-stratification/iterative-stratification-master/LICENSE
/kaggle/input/iterative-stratification/iterative-stratification-master/.gitignore
/kaggle/input/

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor
from scipy.special import expit
from sklearn.metrics import log_loss
from torch.utils.data import DataLoader

In [4]:
class LogitsLogLoss(Metric):

    def __init__(self):
        self._name = 'logits_ll'
        self._maximize = False

    def __call__(self, y_true, y_pred):
        aux = (1 - y_true) * np.log(1 - expit(y_pred) + 1e-15) + y_true * np.log(expit(y_pred) + 1e-15)
        return np.mean(-aux)

In [5]:
def seed_everything(seed, use_cuda=False):
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

    if use_cuda:
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [6]:
def preprocess(df):
    df['cp_dose'] = df['cp_dose'].map({'D1': 0, 'D2': 1})
    df['cp_time'] = df['cp_time'].map({24: 0, 48: 1, 72:2})
    df = df[df['cp_type']  != 'ctl_vehicle'].reset_index(drop=True)
    return df


def prepare_data(dir):
    X_train = pd.read_csv(f'{dir}/train_features.csv')
    Y_train = pd.read_csv(f'{dir}/train_targets_scored.csv')
    X_test = pd.read_csv(f'{dir}/test_features.csv')
    ss = pd.read_csv(f'{dir}/sample_submission.csv')

    train = X_train.merge(Y_train, on='sig_id')
    Y_train_stub = train.loc[:, Y_train.columns]
    
    train = preprocess(train)
    X_test = preprocess(X_test).drop(['cp_type'], axis=1)

    X_train = train.loc[:, X_train.columns].drop(['sig_id', 'cp_type'], axis=1)
    Y_train = train.loc[:, Y_train.columns]

    return X_train, Y_train, Y_train_stub, X_test, ss

In [7]:
EPOCHS = 200
BATCH_SIZE = 1024
LEARNING_RATE = 2e-2
WEIGHT_DECAY = 2e-5
SEED = 42
NUM_FOLDS = 10

In [8]:
def train_tab_net(fold, X_test, X_train, Y_train, X_val, Y_val, train_size, val_idx, out_size):
    oof = np.zeros((train_size, out_size))
    tabnet_params = dict(
                        n_d=32,
                        n_a=32,
                        n_steps=1,
                        gamma=1.7,
                        lambda_sparse=0,
                        optimizer_fn=torch.optim.Adam,
                        optimizer_params=dict(lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY),
                        scheduler_params=dict(mode='min',
                                              patience=5,
                                              min_lr=1e-5,
                                              factor=0.9),
                        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                        mask_type='entmax',
                        seed=SEED,
                        verbose=10
                    )

    model = TabNetRegressor(**tabnet_params)

    model.fit(
        X_train=X_train, 
        y_train=Y_train,
        eval_set = [(X_val, Y_val)],
        eval_name=['val'],
        eval_metric=[LogitsLogLoss],
        max_epochs=EPOCHS,
        patience=20,
        batch_size=BATCH_SIZE,
        virtual_batch_size=128,
        num_workers=1,
        drop_last=False,
        loss_fn=nn.BCEWithLogitsLoss()
    )

    oof[val_idx] = expit(model.predict(X_val))
    Y_pred = expit(model.predict(X_test))

    return np.min(model.history['val_logits_ll']), Y_pred, oof

def run_msk_fold_cv(X_train, Y_train, Y_train_stub, X_test, ss, num_folds, device):
    running_loss = 0
    Y_pred = np.zeros((X_test.shape[0], Y_train.shape[1] - 1))
    mskf = MultilabelStratifiedKFold(n_splits=num_folds, shuffle=True, random_state=0)
    oof = np.zeros((X_train.shape[0], Y_train.shape[1] - 1))

    for fold, (trn_idx, val_idx) in enumerate(mskf.split(X_train, Y_train)):
        fold_X_train = X_train.loc[trn_idx, :].to_numpy()
        fold_Y_train = Y_train.loc[trn_idx, :].drop('sig_id', axis=1).to_numpy()
        fold_X_val = X_train.loc[val_idx, :].to_numpy()
        fold_Y_val = Y_train.loc[val_idx, :].drop('sig_id', axis=1).to_numpy()
        
        print(f'Fold: {fold + 1}')

        fold_loss, fold_Y_pred, fold_oof = train_tab_net(fold, X_test.drop('sig_id', axis=1).to_numpy(), 
                                                         fold_X_train, fold_Y_train,
                                                         fold_X_val, fold_Y_val,
                                                         X_train.shape[0], val_idx, 
                                                         Y_train.shape[1] - 1)
        
        Y_pred += fold_Y_pred
        oof += fold_oof
        running_loss += fold_loss
        
    Y_pred /= num_folds
    oof /= num_folds
    cv_loss = running_loss / num_folds

    oof_Y_pred = Y_train.copy()
    oof_Y_pred.iloc[:, 1:] = oof
    oof_Y_pred = Y_train_stub.loc[:, ['sig_id']].merge(oof_Y_pred, on='sig_id', how='left').fillna(0)

    Y_true = Y_train_stub.iloc[:, 1:].values
    oof_Y_pred = oof_Y_pred.iloc[:, 1:].values

    cv_score = 0

    for i in range(oof_Y_pred.shape[1]):
        cv_score += log_loss(Y_true[:, i], oof_Y_pred[:, i])

    cv_score /= oof_Y_pred.shape[1]

    print(f'CV loss (ctl_vechile excluded): {cv_loss:.6f}')
    print(f'CV loss: {cv_score:.6f}')

    test_Y_pred = X_test.loc[:, ['sig_id']].merge(ss, how='left', on=['sig_id'])
    test_Y_pred.iloc[:, 1:] = Y_pred
    test_Y_pred = ss.loc[:, ['sig_id']].merge(test_Y_pred, on='sig_id', how='left').fillna(0)

    return test_Y_pred

In [9]:
def run_net():
    use_cuda = False
    device = ('cuda' if torch.cuda.is_available() else 'cpu')
    Y_pred = None

    if device == ('cuda'):
        use_cuda = True

    seed_everything(SEED, use_cuda)

    X_train, Y_train, Y_train_stub, X_test, ss = prepare_data('../input/lish-moa')

    Y_pred = run_msk_fold_cv(X_train, Y_train, Y_train_stub, X_test, ss, NUM_FOLDS, device)

    return Y_pred

In [10]:
sub = run_net()

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Fold: 1
Device used : cuda
epoch 0  | loss: 0.32664 | val_logits_ll: 0.03469 |  0:00:02s
epoch 10 | loss: 0.01915 | val_logits_ll: 0.0212  |  0:00:13s
epoch 20 | loss: 0.01778 | val_logits_ll: 0.0201  |  0:00:25s
epoch 30 | loss: 0.01731 | val_logits_ll: 0.01973 |  0:00:37s
epoch 40 | loss: 0.01709 | val_logits_ll: 0.01767 |  0:00:49s
epoch 50 | loss: 0.01677 | val_logits_ll: 0.01868 |  0:01:01s
epoch 60 | loss: 0.0168  | val_logits_ll: 0.01724 |  0:01:13s
epoch 70 | loss: 0.01658 | val_logits_ll: 0.01718 |  0:01:24s
epoch 80 | loss: 0.01643 | val_logits_ll: 0.01703 |  0:01:36s
epoch 90 | loss: 0.01628 | val_logits_ll: 0.01695 |  0:01:48s
epoch 100| loss: 0.01624 | val_logits_ll: 0.01694 |  0:02:01s
epoch 110| loss: 0.01602 | val_logits_ll: 0.01686 |  0:02:13s
epoch 120| loss: 0.01593 | val_logits_ll: 0.01694 |  0:02:25s
epoch 130| loss: 0.01576 | val_logits_ll: 0.017   |  0:02:36s
epoch 140| loss: 0.01565 | val_logits_ll: 0.01693 |  0:02:48s

Early stopping occurred at epoch 148 with 

In [11]:
sub.to_csv('submission.csv', index=False)